In [1]:
# import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys, re, json
from konlpy.tag import Okt 
from collections import Counter 

import warnings
warnings.filterwarnings('ignore')

# load raw data from csv file
rawData=pd.read_csv('홈인테리어.csv')

# load stop words
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()

#load senti words
sentiwordsfile = open('SentiWord_info.json', encoding='utf-8-sig', mode='r')
sentiwords = json.load(sentiwordsfile)

# 명사 형태소 추출 함수
okt = Okt()  

#define functions
def applyRegularExpression(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
    result = hangul.sub('', text)  # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
    return result

def countWords(_data, range):
    if range is None:
        range = len(_data)
    
    data = _data['text'].dropna(axis=0)

    corpus = "".join(data)
    # print('띄어쓰기 없앰', corpus)

    # 문장부호 없애기
    nouns = okt.nouns(applyRegularExpression(corpus))
    #print('문장부호 없앰', nouns)

    no_stop_words = [x for x in nouns if x not in stopwords if len(x) > 1]
    result = Counter(no_stop_words[0:range])
    return result

def countWordsByReview(reviews, range):
    if range is None:
        range = len(reviews)
    for row in data[0:range]:
        result = Counter(row)
    return result

def getScoresFromReviews(reviews, range):
    if range is None:
        range = len(reviews)
    # 리뷰 없는 행 제거
    data = reviews['text'].dropna(axis=0)

    # 리뷰별로 로직 적용
    for row in data[0:range]:
        # row 는 리뷰 하나다
        # 띄어쓰기 없애기
        corpus = "".join(row)
        # print('띄어쓰기 없앰', corpus)

        # 문장부호 없애기
        nouns = okt.nouns(applyRegularExpression(corpus))
        #print('문장부호 없앰', nouns)

        no_stop_words = [x for x in nouns if x not in stopwords if len(x) > 1]
        #print('중성명사 없앰', no_stop_words)

        #리뷰별로 점수 초기화
        score = 0

        for word in no_stop_words:
            #각 단어의 점수 계산
            wordscore = getScoreByWord(word)
            # 리뷰 내 모든 단어의 점수 합산
            score += int(getScoreByWord(word))

        if(score != 0):
            # 0 아니면 로그찍기
            #전체리뷰보기
            #print(row + '...', score)
            # 앞 10글자만 보기
            print(row[0:10] + '...', score)

def getScoreByWord(wordname):
    score = 0
    for i in range(0, len(sentiwords)):
        if sentiwords[i]['word_root'] == wordname:
            result = sentiwords[i]['polarity']
            score = int(result)
            # print('점수 : ' + wordname + ' : ' + str(score))
            
    return score

In [4]:
countWords(rawData, 200).most_common(10)    

[('콘센트', 6),
 ('소리', 5),
 ('하나', 4),
 ('개월', 4),
 ('쿠팡', 4),
 ('물건', 3),
 ('부분', 3),
 ('오늘', 3),
 ('냄새', 3),
 ('멀티탭', 3)]

In [5]:
getScoresFromReviews(rawData, 100)

이딴 쓰레기도 물건... -1
별점 하나도 주기 ... 2
와...
이거 별 ... -1
터졌어요ㅠㅠ  쿠팡... -4
오늘 받았습니다
고... -3
싼 맛에 사지 마세... 1
써바야겟지만 집에 ... -2
한달째 쓰는중에 오... -2
이런 후기 거의.안... 1
사용한지 2년정도 ... 1
쓰다가 터져서 저세... -1
배송은 빠르고 좋았... -1
멀티탭 그동안 여기... -3
별하나도 아까워요 ... -2
전기주전자 에어프라... -1
황당해서 첨으로 리... -4
별로예요
원래 후기... -2
전 아무것도 안했는... -1
헐~   접촉이  ... -1
배소오고나서 콘센트... -2
전선에서 플라스틱탄... -2
사용한지 얼마되지 ... -2
냉장고하나, 드라이... -3
두달 전에 구매했는... 1
한개가 접촉 불량인... -2
멀티탭 한개가 계속... -2
디자인은 무난한데 ... -2
4구중 1구가 안됨... -1
불량이네요 역시 싼... -2
멀티탭 마지막 한개... -4
일주일만에 고장났어... -1
전원을 연결해도 불... -1
합선이 났는지
펑 ... -2
둘중에 하나가 불량... -2
심하게 손상됨 전기... -1
히터 한번 틀었는데... -2
집에 전기렌지, 전... -3
한달만에 지지직 소... -1


In [6]:
#countWordsByReview(rawData,100)

In [ ]:
'''
from sklearn.feature_extraction.text import CountVectorizer

def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 정규 표현식 처리
    result = hangul.sub('', text)
    okt = Okt()  # 형태소 추출
    nouns = okt.nouns(result)
    nouns = [x for x in nouns if len(x) > 1]  # 한글자 키워드 제거
    nouns = [x for x in nouns if x not in stopwords]  # 불용어 제거
    return nouns

vect = CountVectorizer(tokenizer = lambda x: text_cleaning(x))
bow_vect = vect.fit_transform(cleaned.tolist())
word_list = vect.get_feature_names()
count_list = bow_vect.toarray().sum(axis=0)
'''

In [ ]:
'''from sklearn.feature_extraction.text import TfidfTransformer

#단어의 중요도 찾아내는 알고리즘
tfidf_vectorizer = TfidfTransformer()
tf_idf_vect = tfidf_vectorizer.fit_transform(bow_vect)
'''

In [ ]:
#print(tf_idf_vect.shape)